In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas
from scipy.stats.kde import gaussian_kde
from numpy import linspace
import statistics 
import math
from scipy.stats import multinomial
from scipy.stats import dirichlet
from scipy.special import gamma


In [2]:
def dch(k1, k2, k3, alphacurr1, alphacurr2, alphacurr3):
    
    beta_k = gamma(k1)*gamma(k2)*gamma(k3)/(gamma(k1+k2+k3))
    S =0
    for gene in range(samp):
        S = S + np.log( alphacurr1[gene]**(k1-1)* alphacurr2[gene]**(k2-1)* alphacurr3[gene]**(k3-1)/beta_k)
        return S

    
def alphr(a1, a2, a3, prop_a1, prop_a2, prop_a3):
    beta_a = gamma(prop_a1)*gamma(prop_a2)*gamma(prop_a3)/(gamma(prop_a1 + prop_a2 + prop_a3)) 
    alr = np.log(a1**(prop_a1-1)*a2**(prop_a2-1)*a3**(prop_a3-1)) - np.log(beta_a)
    return alr


def rpa(ar1, ar2, ar3, k1, k2, k3, c, nu, d1, d2, d3):
    de = [d1,d2,d3]
    a = [ar1, ar2, ar3]
    m = np.dot(de,a)
    ex = ((-0.5)*c *(r[nu] - m)**2 / (r[nu]**2 + m**2))
    A = np.log((m*(r[nu] + m)/(r[nu]**2 + m**2)**1.5))+ ex + np.log(ar1**(k1-1)*ar2**(k2-1)*ar3**(k3-1))
    return A

def dirich(ka1,ka2,ka3):
    
    c1 = np.random.gamma(ka1,1,1)
    c2 = np.random.gamma(ka2,1,1)
    c3 = np.random.gamma(ka3,1,1)
    
    xc1 = c1/(c1+c2+c3)
    xc2 = c2/(c1+c2+c3)
    xc3 = c3/(c1+c2+c3)
        
    return [xc1, xc2, xc3]       

    
    

In [3]:
dir = np.random.dirichlet([10,6,3],1000000).T
uni1 = np.random.randint(2, size=1000000)
uni2 = np.random.randint(2, size=1000000)
uni3 = np.random.randint(2, size=1000000)
mu = np.zeros(1000000); r= np.zeros(1000000)
for gen in range(1000000):
    mu[gen] = dir[0,gen]*uni1[gen]+dir[1,gen]*uni2[gen]+dir[2,gen]*uni3[gen]
    r[gen] = np.random.normal(mu[gen],.1*mu[gen],1)

In [4]:
import random
import time
st = time.time()
num = 10000
samp = 1000000
K1 = np.zeros(num); K2 = np.zeros(num); K3 = np.zeros(num)

C = np.zeros(num)

A1 = np.zeros(samp); A2 = np.zeros(samp); A3 = np.zeros(samp)

alphacurr1 = np.zeros(samp);alphacurr2 = np.zeros(samp);alphacurr3 = np.zeros(samp)

B1 = np.zeros(7); B2 = np.zeros(7); B3 = np.zeros(7)

k1 = 12;k2 = 16; k3 = 14



acs = 0
acc = 0
nu0 = .1
c0 = 1000
ni = samp
c = .1
ac = 10
#Alphacurr = np.random.dirichlet([5,3,2],10).transpose()
alphacurr1.fill(0.5); alphacurr2.fill(0.3); alphacurr3.fill(0.2)
 
for priork in range(num):
    
    
    for alp in range(samp):
        U = .009
        
        ac1 = alphacurr1[alp]
        ac2 = alphacurr2[alp]
        ac3 = alphacurr3[alp]
            
        prop = np.random.dirichlet([(ac1/U) , (ac2/U) , (ac3/U)], 1).transpose()
        prop_a1 = prop[0]
        prop_a2 = prop[1]
        prop_a3 = prop[2]
        
        
        
       
        p_num = rpa(prop_a1, prop_a2, prop_a3, k1, k2, k3, c, alp,uni1[alp],uni2[alp],uni3[alp])
        p_den = rpa(ac1, ac2, ac3, k1, k2, k3, c, alp,uni1[alp],uni2[alp],uni3[alp])
        alpha_num = alphr(ac1, ac2, ac3, prop_a1/U, prop_a2/U, prop_a3/U)
        alpha_denom = alphr(prop_a1, prop_a2, prop_a3, ac1/U, ac2/U, ac3/U) 
        
        r_alpha = (p_num + alpha_num ) - (p_den + alpha_denom)
        
        
        if(np.log(np.random.uniform(0, 1 , 1)) < r_alpha):
            
            alphacurr1[alp] = prop_a1
            alphacurr2[alp] = prop_a2
            alphacurr3[alp] = prop_a3
            acc = acc+1
        else:
            alphacurr1[alp] = ac1
            alphacurr2[alp] = ac2
            alphacurr3[alp] = ac3
      

    G = (nu0 * c0**2)
    for g in range(samp):
        
        de = [uni1[g],uni2[g],uni3[g]]
        a = [alphacurr1[g], alphacurr2[g], alphacurr1[g]]
        m = np.dot(de,a)
        G = G + ((r[g] - m)**2 / (r[g]**2 + m**2))

    c = (np.random.gamma((nu0 + ni)/2 , ((G)/2), 1))   
    C[priork] = 1 / np.sqrt(c)

   
    
    
    Uk1 = .9
    
    t = np.random.uniform(k1 - Uk1,k1 + Uk1, 1)
    if(t<0):
        prop_K1 = -t
    else:
        prop_K1 = t
        
        
        
    Uk2 = .9
    
    t2 = np.random.uniform(k2 - Uk2,k2 + Uk2, 1)
    if(t2<0):
        prop_K2 = -t2
    else:
        prop_K2 = t2
        
        
    Uk3 = .9
    
    t3 = np.random.uniform(k3 - Uk3,k3 + Uk3, 1)
    if(t3<0):
        prop_K3 = -t3
    else:
        prop_K3 = t3
        
    rp = dch(prop_K1, prop_K2, prop_K3, alphacurr1, alphacurr2, alphacurr3)
    rc = dch(k1, k2, k3, alphacurr1, alphacurr2, alphacurr3)
    kp1 = ((-1*prop_K1)) + ((-2*prop_K2)) +((-3*prop_K3))
    kc1 = ((-1*k1)) + ((-2*k2)) + ((-3*k3))
        
    
    a1 = ( rp + kp1 )-( rc + kc1 )
    
    if(np.log(np.random.uniform(0, 1 , 1)) < a1):
        
        k1 = prop_K1
        k2 = prop_K2
        k3 = prop_K3
        acs = acs + 1
    else:
        k1 = k1
        k2 = k2
        k3 = k3
        
    
    
    K1[priork] = k1
    K2[priork] = k2
    K3[priork] = k3
        
        

et = time.time()  
print((et-st)/60)

/home/anik/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/home/anik/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
/home/anik/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars
/home/anik/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/home/anik/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in double_scalars


KeyboardInterrupt: 

In [5]:
priork

108

In [ ]:
x1 =[100,200,300,400,500]
y1 = [8.6,8.9,9.6,10.05,10.8]
y2 = [6.9, 14.1, 21.5, 28.8, 36.2]
y3 = [1.7*60, 3.4*60, 5.2*60, 6.8*60, 8.8*60]
plt.xlim(90,550)
plt.plot(x1,y1,marker='o') 
plt.plot(x1,y2,marker='+') 
#plt.plot(x1,y3,marker='*') 
plt.xlabel("Data size")
plt.ylabel("Time (sec)")
  
# Function add a legend   
plt.legend(["GPU C++ code", "CPU C++ code", "CPU Python code"], loc ="upper left") 

In [ ]:
y3


In [ ]:
numa = int((((10*num)-(.3*10*num))/100) + 1)

alpha_1 = np.zeros(numa); alpha_2 = np.zeros(numa); alpha_3 = np.zeros(numa);
for thin in range(numa):
    alpha_1[thin] = af1[int(.3*10*num - 1)+100+thin]
    alpha_2[thin] = af2[int(.3*10*num - 1)+100+thin]
    alpha_3[thin] = af3[int(.3*10*num - 1)+100+thin]
    #Cth[thin] = C[thin+100+98900]
  

In [ ]:
plt.plot(af1)

In [ ]:
statistics.mean(alpha_1)

In [ ]:
plt.plot(alpha_2)

In [ ]:
plt.plot(K3)

In [ ]:
x_i1 = np.zeros(numa);x_i2 = np.zeros(numa);x_i3 = np.zeros(numa)
xm_1 = np.zeros(numa); xm_2 = np.zeros(numa);xm_3 = np.zeros(numa)
x1 = np.linspace(0,1,numa)
x2 = np.linspace(0,1,numa)
x3 = np.linspace(0,1,numa)
stda = np.std(alpha_1)
stdb = np.std(alpha_2)
stdc = np.std(alpha_3)
z1 = np.zeros(len(x1))
C1 = (stda/(numa**(1/(3+4))))
C2 = (stdb/(numa**(1/(3+4))))
C3 = (stdc/(numa**(1/(3+4))))
for loop in range(numa):
    for loop2 in range(numa):
        x_i1[loop2]=x1[loop]-alpha_1[loop2]
        x_i2[loop2]=x2[loop]-alpha_2[loop2]
        x_i3[loop2]=x3[loop]-alpha_3[loop2]
    xm_1[loop] = np.mean(np.exp(-.5*(x_i1/C1)**2))*(2*np.pi)**(-1.5)*C1**(-3)
    xm_2[loop] = np.mean(np.exp(-.5*(x_i2/C2)**2))*(2*np.pi)**(-1.5)*C2**(-3)
    xm_3[loop] = np.mean(np.exp(-.5*(x_i3/C3)**2))*(2*np.pi)**(-1.5)*C3**(-3)
    


In [ ]:
plt.plot(x1,xm_1)

In [ ]:
density = gaussian_kde(K3)
xs = np.linspace(0,1,1000)
plt.plot(xs,density(xs))
#plt.plot(K1,np.exp(-K3))
plt.show()
a = (density(xs))
i = list(a).index(max(density(xs))) 
i
xs[i]

In [ ]:
z1 = np.zeros(1000)
for i in range(1000):
    
    z = np.random.dirichlet([K1[i],K2[i],K3[i]],1)
    z1[i] = z[i,0]


In [ ]:
z

In [ ]:
z[0,1]

In [ ]:
np.exp(-.5*(-25+10.62645)**2)/np.sqrt(2*np.pi)

In [ ]:
plt.plot(alpha_1)

In [ ]:
plt.plot(alpha_2)

In [ ]:
plt.plot(alpha_3)

In [ ]:
density = gaussian_kde(alpha_1)
xs = np.linspace(0,1,1000)
plt.plot(xs,density(xs))
#plt.plot(K1,np.exp(-K3))
plt.show()

In [ ]:
density = gaussian_kde(alpha_2)
xs = np.linspace(0,1,1000)
plt.plot(xs,density(xs))
#plt.plot(K1,np.exp(-K3))
plt.show()

In [ ]:
density = gaussian_kde(alpha_3)
xs = np.linspace(0,1,1000)
plt.plot(xs,density(xs))
#plt.plot(K1,np.exp(-K3))
plt.show()

In [ ]:
import numpy as np
def b():
    
    print(u)
    print("back to main")
for i in range(3):
    u = np.random.dirichlet([10,6,3],3).transpose()
    print(u)
    print("calling function")
    b()
   

In [ ]:
z=np.random.dirichlet([1.9,.9,.6],1000000)
z

In [ ]:
z.shape

In [ ]:
gamma(101)

In [ ]:
3**30